In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# changing the permissions of the chromedriver to make it executable
import os
os.chmod("chromedriver", 0o755)

In [ ]:
# installing chrome ?
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y  # Fix any dependency issues
!pip install selenium chromedriver-autoinstaller

In [ ]:
# Scrapping step :
# this script extract the Model (Marque) and subModels (les modeles) ids that we will use in url params to filter our requests when
# scraping listings details

# we need a table explaining the params that we will use and used in filtered urls :
#  ?has_price=true
#  ?barnd=<model number>
#  ?model=<submodel number>
#  ?sep=1 (keep the same order when going through the pagination)
#  ?o=<page number> (to navigate different pages)
# clarify that we combined all these parameters to filter what we need

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import urllib.parse
import time




In [ ]:
# extracting both model and submodel stuff

def setup_driver():
    """Set up Selenium Chrome driver for Colab."""
    chromedriver_path = "./chromedriver"
    if not os.path.exists(chromedriver_path):
        # Install ChromeDriver if not present
        !pip install chromedriver-autoinstaller
        import chromedriver_autoinstaller
        chromedriver_autoinstaller.install()

    options = Options()
    options.add_argument("--headless")  # Run without opening browser window
    options.add_argument("--no-sandbox")  # Required for Colab
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")
    options.binary_location = "/usr/bin/google-chrome"  # Colab Chrome path

    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)
    return driver




In [ ]:
driver = setup_driver()
base_url = "https://www.avito.ma/fr/maroc/voitures_d_occasion-%C3%A0_vendre"
driver.get(base_url)
models_data = []


try :
        wait = wait = WebDriverWait(driver, 15)
#     Click sidebar button
        sidebar_button_selector = "div.sc-19iasnh-1.fNIdCR.horizontalScroll > button"
        sidebar_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, sidebar_button_selector))
        )
        sidebar_button.click()
        time.sleep(1)


        # Click model box to open dropdown
        model_box_selector = "div.sc-jmnnoq-1.cRxZSt > div > div.sc-g1uvop-7.gsTyiq > div > div:nth-child(4) > div"
        model_box = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, model_box_selector))
        )

        print("Model box detected.")
        model_box.click()
        time.sleep(1)



        model_list_selector = "div.sc-1uvojg5-5.kmNuIq > div.sc-1uvojg5-6.gJQYfK > ul > li"
        submodel_list_selector ="div.sc-1uvojg5-6.gJQYfK > ul > li.sc-1uvojg5-11.gtOfny"
        scroll_container_selector = "div.sc-1uvojg5-6.gJQYfK"
        model_names = set()  # Track unique model names
        models_data = []  # Store model data with URLs

        try:
            scroll_container = driver.find_element(By.CSS_SELECTOR, scroll_container_selector)
            max_attempts = 200
            model_index = 0  # Current model position
            no_new_names_count = 0

            # Measure <li> height (assume same for model and submodel)
            elements = driver.find_elements(By.CSS_SELECTOR, model_list_selector)
            li_height = driver.execute_script("return arguments[0].offsetHeight", elements[0]) if elements else 50
            print(f"LI height: {li_height}px")

            print("Tracking models and submodels with pixel-perfect scrolling...")
            while model_index < max_attempts:
                # Scroll to current model index
                driver.execute_script(f"arguments[0].scrollTop = {model_index * li_height}", scroll_container)
                time.sleep(1)

                # Fetch fresh model elements
                elements = driver.find_elements(By.CSS_SELECTOR, model_list_selector)
                if not elements:
                    print("No more models found.")
                    break

                try:
                    # Process first visible model
                    elem = elements[0]
                    span = elem.find_element(By.CSS_SELECTOR, "span.sc-1uvojg5-13.elTbZK")
                    model_name = span.text.strip()

                    if model_name and model_name != "Suggestions" and model_name not in model_names:
                        model_names.add(model_name)

                        # Click model to open submodel list
                        driver.execute_script("arguments[0].click();", elem)
                        time.sleep(1)

                        # Process submodels
                        submodel_index = 0
                        processed_submodels = set()
                        max_submodel_attempts = 100
                        while submodel_index < max_submodel_attempts:
                            # Scroll to submodel index
                            submodel_list = driver.find_element(By.CSS_SELECTOR, "div.sc-1uvojg5-6.gJQYfK")
                            target_scroll = submodel_index * li_height
                            current_scroll = driver.execute_script("return arguments[0].scrollTop", submodel_list)
                            if target_scroll > current_scroll:
                                driver.execute_script(f"arguments[0].scrollTop = {current_scroll + 10}", submodel_list)
                                time.sleep(2)
                            driver.execute_script(f"arguments[0].scrollTop = {target_scroll}", submodel_list)
                            time.sleep(3)
                            print(f"Submodel scroll target: {target_scroll}, actual: {driver.execute_script('return arguments[0].scrollTop', submodel_list)}")

                            submodel_scroll_height = driver.execute_script("return arguments[0].scrollHeight", submodel_list)
                            print(f"Submodel scroll height: {submodel_scroll_height}")



                            if submodel_index * li_height > submodel_scroll_height:
                                print(f"Exceeded submodel scroll height for {model_name} at {submodel_index * li_height}px, breaking.")
                                break

                            # Fetch fresh submodel elements
                            submodel_elements = driver.find_elements(By.CSS_SELECTOR, submodel_list_selector)
                            if not submodel_elements:
                                print(f"No more submodels for {model_name}.")
                                break

                            found_new_submodel = False
                            # here1
                            try:
                                found_new_submodel = False
                                for i, sub_elem in enumerate(submodel_elements):
                                    try:
                                        print("submodel_elements: ", submodel_elements)
                                        try:
                                          sub_span = sub_elem.find_element(By.CSS_SELECTOR, "span.sc-1uvojg5-13.elTbZK")
                                          submodel_name = sub_span.text.strip()
                                        except Exception as e:
                                          print("span doesnt exist submodel-level")
                                          break


                                        print(f"the {i} index Submodel name: {submodel_name}")
                                        if submodel_name and submodel_name != "Suggestions" and submodel_name not in processed_submodels:
                                            processed_submodels.add(submodel_name)
                                            driver.execute_script("arguments[0].click();", sub_elem)
                                            time.sleep(1)
                                            # Capture URL
                                            current_url = driver.current_url
                                            parsed = urllib.parse.urlparse(current_url)
                                            params = urllib.parse.parse_qs(parsed.query)
                                            brand_id = params.get('brand', [None])[0]
                                            model_id = params.get('model', [None])[0]
                                            models_data.append({
                                                'model_name': model_name,
                                                'submodel_name': submodel_name,
                                                'url': current_url,
                                                'brand_id': brand_id,
                                                'model_id': model_id
                                            })
                                            print(f"Captured: {model_name} -> {submodel_name} -> {current_url} (brand_id={brand_id}, model_id={model_id})")
                                            # closing the small box after we collected the data
                                            try:
                                                btn_close_selector = "li.sc-1uvojg5-21.kQGrdj > div.sc-1uvojg5-20.caslNC > div.sc-1uvojg5-7.crzJpY > button.sc-1uvojg5-9.bMARfn"
                                                btn_close = driver.find_element(By.CSS_SELECTOR, btn_close_selector)
                                                driver.execute_script("arguments[0].click();", btn_close)
                                                time.sleep(1)
                                            except:
                                                print(f"No submodel list or close button found for {model_name}, continuing.")
                                            found_new_submodel = True
                                            break
                                    except Exception as e:
                                        print(f"Error processing submodel at index {i} for {model_name}: {e}")
                                        continue
                                if not found_new_submodel:
                                    print(f"No new submodels in current view for {model_name}, scrolling.")
                                    submodel_index += 1
                                else:
                                    # Restore main model scroll position
                                    driver.execute_script(f"arguments[0].scrollTop = {model_index * li_height}", scroll_container)
                                    time.sleep(2)
                                    # Reopen submodel list
                                    elements = driver.find_elements(By.CSS_SELECTOR, model_list_selector)
                                    if elements:
                                        driver.execute_script("arguments[0].click();", elements[0])
                                        time.sleep(2)

                            # here1
                            except Exception as e:
                                print(f"Error processing submodel at index {submodel_index} for {model_name}: {e}")
                                submodel_index += 1
                                continue

                        # Close submodel list
                        try:
                            btn_close_selector = "li.sc-1uvojg5-21.kQGrdj > div.sc-1uvojg5-20.caslNC > div.sc-1uvojg5-7.crzJpY > button.sc-1uvojg5-9.bMARfn"
                            btn_close = driver.find_element(By.CSS_SELECTOR, btn_close_selector)
                            driver.execute_script("arguments[0].click();", btn_close)
                            time.sleep(1)
                        except:
                            print(f"No submodel list or close button found for {model_name}, continuing.")

                    # Restore main model scroll position
                    driver.execute_script(f"arguments[0].scrollTop = {model_index * li_height}", scroll_container)
                    time.sleep(1)

                except Exception as e:
                    print(f"Error processing model at index {model_index}: {e}")
                    model_index += 1
                    continue

                model_index += 1
                # Refresh model list
                driver.execute_script(f"arguments[0].scrollTop = {model_index * li_height}", scroll_container)
                time.sleep(2)
                elements = driver.find_elements(By.CSS_SELECTOR, model_list_selector)
                print(f"Model index: {model_index}, Found {len(elements)} <li>, Total unique models: {len(model_names)}")


                # Stop if no new models
                if not model_names or len(elements) == 0:
                    no_new_names_count += 1
                    if no_new_names_count >= 3:
                        print("No new models after multiple attempts, stopping.")
                        break
                else:
                    no_new_names_count = 0

            # Remove duplicates
            unique_models_data = []
            seen_pairs = set()
            for item in models_data:
                pair = (item['model_name'], item.get('submodel_name'))
                if pair not in seen_pairs:
                    unique_models_data.append(item)
                    seen_pairs.add(pair)

            model_names = sorted(list(model_names))
            print(f"Total unique model names extracted: {len(model_names)}")
            print("Model names:", model_names)
            print("Unique models_data:", unique_models_data)

        except Exception as e:
            print(f"Error tracking models: {e}")
            print("Page source for debugging:")
            print(driver.page_source[:1000])
            print("models_data:", models_data)

except Exception as e:
    print(f"something is not detected {e}")

In [2]:
import pandas as pd

df = pd.read_csv(r"C:\Users\pc\Desktop\school_deep_notes\machine learning\model_and_sub_urls_v2.csv")
df.head()

,model_name,submodel_name,url,brand_id,model_id
0,Mercedes-Benz,190,https://www.avito.ma/fr/maroc/voitures_d_occas...,41,190
1,Mercedes-Benz,207D,https://www.avito.ma/fr/maroc/voitures_d_occas...,41,207d
2,Mercedes-Benz,210,https://www.avito.ma/fr/maroc/voitures_d_occas...,41,210
3,Mercedes-Benz,220,https://www.avito.ma/fr/maroc/voitures_d_occas...,41,220
4,Mercedes-Benz,230,https://www.avito.ma/fr/maroc/voitures_d_occas...,41,230


In [5]:
import pandas as pd

df = pd.read_csv(r"C:\Users\pc\Desktop\school_deep_notes\machine learning\merged_output_cp.csv")


Index(['url', 'title', 'price', 'location', 'brand', 'year', 'transmission',
       'fuel_type', 'kilometrage', 'brand.1', 'model', 'origin', 'first_owner',
       'fiscal_horsepower', 'condition', 'abs', 'airbags', 'electric_windows',
       'remote_central_locking', 'parking_sensors', 'navigation_system/gps',
       'rear_view_camera', 'speed_limiter', 'alloy_wheels', 'esp',
       'air_conditioning', 'cd/mp3/bluetooth', 'sieges_cuir',
       'ordinateur_de_bord', 'sunroof', 'cruise_control', 'door_number'],
      dtype='object')

In [ ]:
df.head()

In [7]:
df.columns

Index(['url', 'title', 'price', 'location', 'brand', 'year', 'transmission',
       'fuel_type', 'kilometrage', 'brand.1', 'model', 'origin', 'first_owner',
       'fiscal_horsepower', 'condition', 'abs', 'airbags', 'electric_windows',
       'remote_central_locking', 'parking_sensors', 'navigation_system/gps',
       'rear_view_camera', 'speed_limiter', 'alloy_wheels', 'esp',
       'air_conditioning', 'cd/mp3/bluetooth', 'sieges_cuir',
       'ordinateur_de_bord', 'sunroof', 'cruise_control', 'door_number'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22639 entries, 0 to 22638
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   url                     22639 non-null  object 
 1   title                   22626 non-null  object 
 2   price                   22626 non-null  object 
 3   location                22626 non-null  object 
 4   brand                   22639 non-null  object 
 5   year                    22626 non-null  object 
 6   transmission            22595 non-null  object 
 7   fuel_type               22626 non-null  object 
 8   kilometrage             22626 non-null  object 
 9   brand.1                 22626 non-null  object 
 10  model                   22626 non-null  object 
 11  origin                  20005 non-null  object 
 12  first_owner             19820 non-null  object 
 13  fiscal_horsepower       22588 non-null  object 
 14  condition               21818 non-null